In [1]:
# Attempt Colab setup if on Colab
try:
  import google.colab
except:
  ALLOW_IO = False
else:
  # Mount Google Drive at data_dir
  #  (for data)
  from google.colab import drive
  from os.path import join
  ROOT = '/content/drive'
  DATA = 'My Drive/phutball'
  drive.mount(ROOT)
  ALLOW_IO = True
  data_dir = join(ROOT, DATA)
  !mkdir "{data_dir}"     # in case we haven't created it already   

  # Pull in code from github
  %cd /content
  github_repo = 'https://github.com/rcharan/phutball'
  !git clone -b master {github_repo}
  %cd /content/phutball
  
  # Point python to code base
  import sys
  sys.path.append('/content/phutball/pytorch-implementation')

  # Updater for library functions changed on local hardware and pushed to github
  #  (circuitous, I know)
  def update_repo():
    !git pull

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/phutball’: File exists
/content
Cloning into 'phutball'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 2050 (delta 36), reused 32 (delta 16), pack-reused 1984
Receiving objects: 100% (2050/2050), 38.53 MiB | 31.95 MiB/s, done.
Resolving deltas: 100% (1287/1287), done.
/content/phutball


In [0]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [3]:
VERSION = "1.5"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  23688      0 --:--:-- --:--:-- --:--:-- 23688
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 1 objects/79.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-1.5-cp36-cp36m-linux_x86_64.whl...
\ [1 files][106.6 MiB/106.6 MiB]                                                
Operation completed over 1 objects/106.6 MiB.                     

In [0]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [0]:
%%capture

%load_ext autoreload
%autoreload 2

import os
import gc
import numpy as np

# Codebase
from lib.model_v3          import TDConway
from lib.off_policy        import EpsilonGreedy
from lib.optim             import AlternatingTDLambda

from lib.training          import training_loop

from lib.utilities         import config, lfilter
from lib.testing_utilities import create_state, visualize_state, boards
from lib.timer             import Timer

from lib.move_selection    import get_next_move_training


# Graphics for visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
plt.ioff()

In [0]:
# Set the device
device = xm.xla_device()

# Memory inspection
def print_memory_usage():
  print('TPU memory inspection not implemented')
def print_max_memory_usage():
  print('TPU memory inspection not implemented')
def garbage_collect():
  gc.collect() # No TPU specific implementation yet


In [0]:
from lib.utilities import product

In [0]:
initial_state = create_state('H10').to(device)
epsilon_greedy = EpsilonGreedy(0.01)

def num_params(model):
  return sum(
      product(t.shape) for t in model.parameters()
  )

In [0]:
model     = TDConway(config).to(device)
optimizer = AlternatingTDLambda(model.parameters(), alpha = 0.05, lamda = 0.9)
print(f'{num_params(model):,d} parameters')
get_next_move_training(initial_state, model, device)
training_loop(model, optimizer, 1, device, off_policy = epsilon_greedy)

1,935,105 parameters
1/1 [==============================] - 0s 5ms/step


In [0]:
from torch.optim import SGD
# version = '0.3.1'
# game_num = '70000'
model     = TDConway(config).to(device)
model     = load(version, game_num, model)

optimizer = AlternatingTDLambda(model.parameters(), alpha = 0.05, lamda = 0.9)

print(f'{num_params(model):,d} parameters')
%timeit get_next_move_training(initial_state, model, device)
%timeit training_loop(model, optimizer, 1, device, off_policy = epsilon_greedy)
# %prun training_loop(model, optimizer, 1, device, off_policy = epsilon_greedy)